In [1]:
import os
%pwd

'd:\\end-to-end-dl-project\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\end-to-end-dl-project'

In [3]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/vedanshgupta606/end-to-end-dl-project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'vedanshgupta606'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '75696140e45e14e44c90129246c954fd067be2c1'

In [4]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri:str
    params_image_size:list
    params_batch_size:int

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            all_params=self.params,
            mlflow_uri="https://dagshub.com/vedanshgupta606/end-to-end-dl-project.mlflow",
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE
        )
        return evaluation_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

        @staticmethod
        def load_model(path : Path) -> tf.keras.Model:
            return tf.keras.models.load_model(path)
        
        def evaluation(self):
            self.model = self.load_model(self.config.path_of_model)
            self._valid_generator()
            self.score = model.evaluate(self.valid_generator)
            self.save_score()

        def save_score(self):
            scores = {"loss": self.score[0], "accuracy": self.score[1]}
            save_json(path=Path("scores.json"), data=scores)

        
        def log_into_mlflow(self):
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            
            with mlflow.start_run():
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(
                    {"loss": self.score[0], "accuracy": self.score[1]}
                )
                # Model registry does not work with file store
                if tracking_url_type_store != "file":

                    # Register the model
                    # There are other ways to use the Model Registry, which depends on the use case,
                    # please refer to the doc for more information:
                    # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-03-16 21:31:00,780: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-16 21:31:00,780: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-16 21:31:00,786: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 25s 3s/step - loss: 0.9398 - accuracy: 0.8345
[2024-03-16 21:31:26,339: INFO: common: json file saved at: scores.json]


2024/03/16 21:31:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-03-16 21:31:30,955: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\vedan\AppData\Local\Temp\tmpva7z1hew\model\data\model\assets
[2024-03-16 21:31:31,706: INFO: builder_impl: Assets written to: C:\Users\vedan\AppData\Local\Temp\tmpva7z1hew\model\data\model\assets]


d:\end-to-end-dl-project\dlenv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/03/16 21:39:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
